### This code works fine but the GUI library has a glitch that it kills kernel second time you run. 
### That's why we run it as a subprocess 

In [3]:
import numpy as np
import open3d as o3d
import PySimpleGUI as sg
import matplotlib.pyplot as plt
import copy
from PIL import Image
from io import BytesIO
import time
import glob

sample_ply_data = o3d.data.DemoCropPointCloud()
tst_pointCloud = o3d.io.read_point_cloud(sample_ply_data.point_cloud_path)
#o3d.visualization.draw_geometries([tst_pointCloud])

def array_to_data(array):
    im = Image.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1,origin=[0, 0, 0])
    mesh_pc= mesh.sample_points_uniformly(number_of_points=1000000, use_triangle_normal=False)
    pointCloud+=mesh_pc
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.radians(RX), np.radians(RY), np.radians((RZ))))
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    #time.sleep(5)
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color


# Cluster Selection

In [ ]:
dat=[]  #to hold cluster's images

clouds = [tst_pointCloud,tst_pointCloud]

for current_cloud in clouds:
        color = fetch_cloud_image(current_cloud, RX=120, RY=0, RZ=220)  #different point clouds to be viewed
        dat.append(color)

In [ ]:
def Cluster_selection_gui(dat):
    
    selected_PC = [] #will hold index values of selected point clouds.
    selected_motion = 0  #default auto

    file_list_column = [[sg.Text("Select all valid Clusters"),]]
    for i in range (len(dat)):
        #print(i)
        file_list_column.append( [sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i), image_source=dat[i], 
                                     image_size = (None, None), image_subsample = 5, )] )

    buttons_layout = [
        [sg.Radio(text = "Auto select motion Path",default = True, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_Auto',
                  auto_size_text = True, group_id=0, pad = ((5, 0), (0, 8)))],
        
        [sg.Radio(text = "Motion along X-axis",default = False, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_X',
                  auto_size_text = True, group_id=0 )],
        
        [sg.Radio(text = "Motion along Y-axis",default = False, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_Y',
                  auto_size_text = True, group_id=0 ),          
         sg.Button(button_text = "Cancel",enable_events = True, tooltip ='First Cluster will be auto-selected', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((60, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "OK",enable_events = True, disabled = True, tooltip ='Select atleast one cluster',
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]
                 ]
    
    
    
    image_viewer_column = [
        [sg.Text("Preview")],
        [sg.Text(size=(40, 1), key="-TOUT-")],
        [sg.Image(key="img",source=dat[0])], [sg.HSeparator(pad=(0,10))], [sg.Column(buttons_layout)]  ]


    layout = [ [ sg.Column(file_list_column,size = (None, None), 
                           scrollable=True, vertical_scroll_only=True,expand_y=True ), 
                sg.VSeparator(), 
                sg.Column(image_viewer_column), ] 
             ]

    window = sg.Window("Pointcloud Browser", layout)

    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Cancel" or event == sg.WIN_CLOSED:
            selected_PC = [0] #default
            selected_motion = 0 #default
            break

        elif event == "Rad_Auto":
            selected_motion = 0 #default
        elif event == "Rad_X":  #curve along Y-axis and motion along X axis
            selected_motion = 2 
        elif event == "Rad_Y":  #curve along X-axis and motion along Y axis
            selected_motion = 3 

        elif event_name == "but":
            try:
                window["img"].update(source=dat[event_idx])
            except:
                print("Error")
                pass
            #if event_idx == 0:

            if (event_idx in selected_PC) == True:
                available_index = selected_PC.index(event_idx)
                del_item = selected_PC.pop(available_index)
                window["but"+str(event_idx)].update(text = "", image_data=dat[event_idx], image_subsample = 5)
                selected_PC = sorted(selected_PC)
                if len(selected_PC)==0:
                    window['OK'].set_tooltip("Select atleast one cluster")
                    window["OK"].update(disabled = True)
                #print(selected_PC)
            else:
                selected_PC.append(event_idx)
                window["but"+str(event_idx)].update(text = "SELECTED",  image_subsample = 5)
                window['OK'].set_tooltip("You may Proceed!")
                window["OK"].update(disabled = False)               
                selected_PC = sorted(selected_PC)
                #print(selected_PC)

        elif event == "OK":

            #print("The current selection includes:",selected_PC)
            break

    window.close()

    return selected_PC,selected_motion


new_one = Cluster_selection_gui(dat)
new_one

# Front screen

In [ ]:
def front_gui():
    selected_option = 0 #will hold which value was selected.

    radio_layout = [
        [sg.Radio(text = "Re-run the previously saved motion",default = True, enable_events = True, 
                  tooltip ="Robot's motion will depend on previously saved coordinates.",key = 'Replay',
                  auto_size_text = True, group_id=0 , font=("gothic", 12))],

        [sg.Radio(text = "Scan the Object (Manual mode)",default = False, enable_events = True, 
                  tooltip ='User will provide inputs at various places',key = 'Manual',
                  auto_size_text = True, group_id=0, font=("gothic", 12) )],

        [sg.Radio(text = "Scan the Object (Automatic mode)",default = False, enable_events = True, 
                  tooltip ='Program will iterate over manual-mode once it is setup.',key = 'Auto',
                  auto_size_text = True, group_id=0, font=("gothic", 12) )],
        [sg.Radio(text = "Change Default settings.",default = False, enable_events = True, 
                  tooltip ='Change the program presets and finetune default parameters.',key = 'Settings',
                  auto_size_text = True, group_id=0, font=("gothic", 12) )]
                 ]

    buttons_layout = [[sg.Button(button_text = "Exit",enable_events = True, tooltip ='Exit the program', 
                                     size = (10, 2),key = 'Exit', image_source=None,
                                     pad = ((60, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "OK",enable_events = True, disabled = False, tooltip ="Let's Go!",
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]]

    image_viewer_column = [
        [sg.Text("What would you like to do?", size=(40, 1), auto_size_text = True , font=("gothic", 14))],
        [sg.Column(radio_layout)],
        [sg.HSeparator(pad=(0,10))],
        [sg.Column(buttons_layout)],
    ]



    layout = [image_viewer_column]

    window = sg.Window("Cognitive Motion Planning", layout)


    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            print(event_name, event_idx)
        except:
            pass

        if event == "Exit" or event == sg.WIN_CLOSED:
            selected_option = -1 #default Exit the program
            break

        elif event == "Replay":
            selected_option = 0 #Replay coords
        elif event == "Manual":  #Manually select stuff
            selected_option = 1 
        elif event == "Auto":  #Auto select stuff
            selected_option = 2
        elif event == "Settings":  #Change settings
            selected_option = 3 
        elif event == "OK":

            #print("The current selection includes:",selected_option)
            break

    window.close()


# Settings

In [ ]:
def Settings_gui(data):
    selected_option = 0 #will hold which value was selected.

    samples = data[0] #1
    spacing = data[1] #0.01
    offset_y = data[2] #0.13
    offset_z = data[3] #0.0
    trim_base = data[4] #0.05
    manual_offset = data[5] #0.0
    Cluster_centered = data[6] #True

    if Cluster_centered == True:
        Cluster_idx = 0
    else:
        Cluster_idx = data[7] #200

    Cluster_discard = data[8] #0 #disabled else put number of pts as threshold
    eps = data[9] #0.05
    min_points = data[10] #10
    Cluster_trim = data[11] #0.01
    TGT_coord_Samples = data[12] #3
    TGT_final_trim = data[13] #0.8 #add max. euclidean distance away from robot's base (less than max reach of EEF) or z values less than...
    TGT_reverse = data[14] #True
    TGT_preview = data[15] #True
    z_offset = data[16] #0.3
    coord_skip = data[17] #3
    TGT_motion_delay = data[18] #0.1
    TGT_save = data[19] #True
    Dbug = data[20]

    font_small = ("gothic", 12)
    font_heading = ("gothic", 14)

    text_layout = [[sg.Text("Point Cloud Settings", size=(60, 2), font=font_heading)],


                   [sg.Text("Number of pointcloud samples:", size=(60, 1), auto_size_text = True 
                            , font=font_small, expand_y=True), sg.Input(default_text = samples, size=(10, 1)
                            ,key = "PCD_Samples", pad = (0,0), focus=True
                            , tooltip = "number of samples to take before generating Point cloud" )],

                   [sg.Text("Set Point cloud Spacing", size=(60, 1)
                            , font=font_small), sg.Input(default_text = spacing
                    ,size=(10, 1),key = "PCD_spacing", pad = (0,0), tooltip = "PCD resolution in Meters" )],

                   [sg.Text("Y and Z offsets:", size=(60, 1), auto_size_text = True 
                            , font=font_small, expand_y=True), sg.Input(default_text = offset_y, size=(4, 1)
                                , key = "PCD_offset_y", pad = (0,0), tooltip = "Y offset in Meters" )
                   , sg.Input(default_text = offset_z, size=(4, 1), key = "PCD_offset_z", pad = (10,0)
                              , tooltip = "Z offset in Meters" )],

                   [sg.Text("Length from base to trim background from point cloud:", size=(60, 1)
                            , font=font_small, expand_y=True), sg.Input(default_text = trim_base, 
                                                             size=(10, 1),key = "trim_base", pad = (0,0)
                    , tooltip = "Trim any background points from Point cloud (in Meters)" )], 

                   [sg.Text("Offset distance between robot's base and Object", size=(60, 1)
                            , font=font_small, expand_y=True), sg.Input(default_text = manual_offset, 
                                                             size=(10, 1),key = "manual_offset", pad = (0,0) 
                    , tooltip = "useful if the object is not at same level as robot's base or in vertical scan (in Meters)")],



                   [sg.HSeparator(pad=(0,10))],

                   [sg.Text("Clustering settings ", size=(60, 2), auto_size_text = True 
                            , font=font_heading, expand_y=True)],

                   [sg.Text("Use Center of the object to generate Target point cloud points:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = Cluster_centered, key ="Cluster_centered",enable_events=True
                                  ,pad=((0, 0),(0,10)))],

                   [sg.Text("Select index to generate target point cloud:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Input(default_text = Cluster_idx, size=(10, 1),key = "Cluster_idx", pad = (0,0),disabled = Cluster_centered
                   , tooltip = "set index value to generate target point cloud, used only if centered tag is unchecked" )],


                   [sg.Text("Set threshold to discard Point clouds with less points. (0 will disable this feature)"
                            , size=(60, 2), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Input(default_text = Cluster_discard, size=(10, 1),key = "Cluster_discard", pad = (0,0)
                               , tooltip = "Minimum number of points required to be accepted as a cluster." )],

                   [sg.Text("Set Clustering parameters: Epsillon and Minimum number of points:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    ,sg.Input(default_text = eps, size=(4, 1)
                              , key = "Cluster_eps", pad = (0,0), tooltip = "Epsillon value" )
                   , sg.Input(default_text = min_points, size=(4, 1)
                              , key = "Cluster_min_pts", pad = (10,0), tooltip = "Minimum number of points" )],

                   [sg.Text("Trim ends of generated point cloud:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Input(default_text = Cluster_trim, size=(10, 1),key = "Cluster_trim", pad = (0,0)
                               , tooltip = "Trim the edges of point cloud to remove outliers (in Meters)" )],


                   [sg.HSeparator(pad=(0,10))],

                   [sg.Text("Target Generation and Execution settings ", size=(60, 2), auto_size_text = True 
                            , font=font_heading, expand_y=True)],               

                   [sg.Text("Generated coordinate sets to reduce ghost target anomaly:", size=(60, 1)
                            , font=font_small, expand_y=True), sg.Input(default_text = TGT_coord_Samples
                    , size=(10, 1),key = "TGT_coord_Samples", pad = (0,0), tooltip = "reduces ghost targets, default: 3" )],

                   [sg.Text("Drop targets that are below certain threshold (use this to remove invalid targets that are out of reach or dangerous for the robot.)", 
                            size=(60, 2), font=font_small, expand_y=True)
                    , sg.Input(default_text = TGT_final_trim, size=(10, 1),key = "TGT_final_trim", pad = (0,0)
                              , tooltip = "remove all those targets that are in unsafe zone")],

                   [sg.Text("Reverse the order of the Targets:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_reverse, key ="TGT_reverse",pad=((0, 0),(0,5))
                                  , tooltip = "Reverse the order ot Target execution")],

                   [sg.Text("Publish preview targets:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_preview, key ="TGT_preview",pad=((0, 0),(0,5))
                                  , tooltip = "Preview Generated Targets in Rviz")],                

                   [sg.Text("Distance between Camera and Object:"
                            , size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Input(default_text = z_offset, size=(10, 1),key = "TGT_Z_offset", pad = (0,0)
                   , tooltip = "This is Z distance from camera to object. Make sure to include EEF's distance too (in Meters)" )],


                   [sg.Text("Number of coordinate targets to skip while moving", size=(60, 1), font=font_small)
                    , sg.Input(default_text = coord_skip, size=(10, 1),key = "TGT_coord_skip", pad = (0,0)
                   , tooltip = "Used to scale down the total number of targets available:" )],

                   [sg.Text("Delay in robot's step motion", size=(60, 1), font=font_small)
                    , sg.Input(default_text = TGT_motion_delay,size=(10, 1),key = "TGT_motion_delay", pad = (0,0)
                   , tooltip = "Step delay between each target execution (in Sec)" )],

                   [sg.Text("Save Targets?", size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_save, key ="TGT_save",pad=((0, 0),(0,5))
                  , tooltip = "Save Generated Targets for later use.")], 
                   
                   [sg.Text("Debug Mode?", size=(60, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = Dbug, key ="Dbug",pad=((0, 0),(0,5))
                  , tooltip = "Setting this will show all generated point clouds, images and steps.")],
                  ]


    buttons_layout = [[sg.Button(button_text = "Cancel",enable_events = True, tooltip ='Exit the program without saving changes.', 
                                     size = (10, 2),key = 'Exit', image_source=None,
                                     pad = ((60, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "Save",enable_events = True, disabled = False, tooltip ="Let's Go!",
                                     size = (10, 2),key = 'Save', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]]

    image_viewer_column = [
        [sg.Column(text_layout)],
        [sg.HSeparator(pad=(0,10))],
        [sg.Column(buttons_layout)],
    ]



    layout = [image_viewer_column]

    window = sg.Window("Settings", layout)


    while True: # Run the Event Loop
        event, values = window.read()
        #print(event, values)
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Exit" or event == sg.WIN_CLOSED:
            selected_option = -1 #default Exit the program
            break

        elif event == "Save":
            selected_option = 1
            samples = window["PCD_Samples"].get()
            spacing = window["PCD_spacing"].get()
            offset_y = window["PCD_offset_y"].get()
            offset_z = window["PCD_offset_z"].get()
            trim_base = window["trim_base"].get()
            manual_offset = window["manual_offset"].get()
            Cluster_centered = window["Cluster_centered"].get()
            Cluster_idx = window["Cluster_idx"].get()
            Cluster_discard = window["Cluster_discard"].get() #disabled else put number of pts as threshold
            eps = window["Cluster_eps"].get()
            min_points=window["Cluster_min_pts"].get()
            Cluster_trim = window["Cluster_trim"].get()
            TGT_coord_Samples = window["TGT_coord_Samples"].get()
            TGT_final_trim = window["TGT_final_trim"].get() #add max. euclideaan...
            TGT_reverse = window["TGT_reverse"].get()
            TGT_preview = window["TGT_preview"].get()
            z_offset = window["TGT_Z_offset"].get()
            coord_skip = window["TGT_coord_skip"].get()
            TGT_motion_delay = window["TGT_motion_delay"].get()
            TGT_save = window["TGT_save"].get()
            Dbug = window["Dbug"].get()

            break

        elif event == "Cluster_centered":

            #print(window["Cluster_centered"].get())
            if window["Cluster_centered"].get() == True:
                window["Cluster_idx"].update(disabled = True)
                window["Cluster_idx"].update(value = "0") #replace this to load previous value

            else:
                window["Cluster_idx"].update(disabled = False)
                window["Cluster_idx"].update(value = "0") 


    window.close()


    return selected_option, samples,spacing,offset_y,offset_z,trim_base,manual_offset,Cluster_centered,Cluster_idx,Cluster_discard,eps,min_points,Cluster_trim,TGT_coord_Samples,TGT_final_trim,TGT_reverse,TGT_preview,z_offset,coord_skip,TGT_motion_delay,TGT_save,Dbug


In [ ]:
data_in = [2,0.01,0.13,0.0,0.05,0.0,True,200,0,0.05,10,0.01,3,0.8,True,True,0.3,3,0.1,True]
new_OP = Settings_gui(data_in)
print(new_OP)


# Manual

In [4]:
def manual_gui(dat,loaded_manual_offset,TGT_save):
    font_small = ("gothic", 12)
    font_heading = ("gothic", 14)
    selected_mode = 0 #will determine the initial pose of the robot
    man_offset = 0.0 
    Exit_flag = 0
    desc = ["Default robot pose with camera facing down towards the object",
            "Robot extended to the highest point and camera facing down towards the object.",
            "Robot facing forward and camera at a very low position. Object right in front of the robot.",
            "Robot facing forward and camera at a higher position. Object right in front of the robot.",
            "Robot and camera looking towards LEFT at a LOW position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot and camera looking towards LEFT at a HIGHER position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot EXTENDED and camera looking towards LEFT at a LOW position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot EXTENDED and camera looking towards LEFT at a HIGHER position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot and camera looking towards RIGHT at a LOW position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot and camera looking towards RIGHT at a HIGHER position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot EXTENDED and camera looking towards RIGHT at a LOW position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot EXTENDED and camera looking towards RIGHT at a HIGHER position. Object is aligned to robot's base (Horizontal distance is 0).",
           ]
    
    while len(desc) < len(dat):
        desc.append("N/A") #add empty descriptions if more imgs added.
    
    
    file_list_column = [[sg.Text("Select a Pose that best describes your case.", font=font_heading),]]
    for i in range (0,len(dat),2):
        #print(i)
        file_list_column.append( [sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i), image_source=dat[i], 
                                     image_size = (None, None), image_subsample = 5, ), 
                                  sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i+1), image_source=dat[i+1], 
                                     image_size = (None, None), image_subsample = 5, )] )

    buttons_layout = [
        [sg.Text("Enter Horizontal distance between robot's base and the Object:", size=(53, 1)
                            , font=font_small, expand_y=True), sg.Input(default_text = 0.0, 
                         size=(10, 1),key = "manual_offset",tooltip = "(in Meters)",disabled = True )],
        
        [sg.Text("", size=(38, 1), auto_size_text = True , font=font_small, expand_y=True), 
         sg.Text("Save Targets?", size=(13, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_save, key ="TGT_save",pad=((0, 0),(0,5))
                  , tooltip = "Save Generated Targets for later use.")],
        
        [sg.Button(button_text = "Cancel",enable_events = True, tooltip ='First Default pose will be auto-selected', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((60, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         sg.Button(button_text = "OK",enable_events = True, disabled = True, tooltip ='Select a Pose first!',
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]
                 ]
    
    
    
    image_viewer_column = [
        [sg.Text("Preview", font = font_heading)],
        [sg.Text("Description: " + desc[0],size=(100, 2),font=font_small, key="-TOUT-")],
        [sg.Image(key="img",source=dat[0])], [sg.HSeparator(pad=(0,10))], [sg.Column(buttons_layout)]  ]


    layout = [ [ sg.Column(file_list_column,size = (None, None), 
                           scrollable=True, vertical_scroll_only=True,expand_y=True ), 
                sg.VSeparator(), 
                sg.Column(image_viewer_column), ] 
             ]

    window = sg.Window("Manual Mode: Initial Pose selector.", layout)

    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Cancel" or event == sg.WIN_CLOSED:
            selected_mode = 0 #default Panda pose
            Exit_flag = 1 #EXIT
            break

        elif event_name == "but":
            try:
                window["img"].update(source=dat[event_idx])
            except:
                #print("Error")
                pass

            window["but"+str(selected_mode)].update(text = "",  image_subsample = 5) #clear last one
            selected_mode = event_idx
            window["but"+str(selected_mode)].update(text = "SELECTED",  image_subsample = 5)
            window["-TOUT-"].update(value = "Description: " + desc[selected_mode])
            window['OK'].set_tooltip("You may Proceed!")
            window["OK"].update(disabled = False)               
            #print(selected_mode)
            
            if selected_mode == 4 or selected_mode == 5 or selected_mode == 8 or selected_mode == 9:
                window["manual_offset"].update(value = loaded_manual_offset, disabled = False, move_cursor_to = "end")
                window["manual_offset"].set_tooltip("Value loaded from saved settings. (in Meters)")
            else:
                window["manual_offset"].update(value = 0.0, disabled = True)
                window["manual_offset"].set_tooltip("Not required. Setting to 0")

        elif event == "OK":
            
            man_offset = window["manual_offset"].get()
            TGT_save = window["TGT_save"].get()
            
            #print("The current selection includes:",selected_mode)
            break

    window.close()

    return selected_mode, man_offset, TGT_save,Exit_flag


dat=[]  #to hold images
loaded_manual_offset = 0.1
TGT_save = True
for f in sorted(glob.iglob("VI_appdata/Robo_object_positions/*")):
    im = Image.open(f)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    dat.append(data)


new_one = manual_gui(dat,loaded_manual_offset,TGT_save)
new_one

(0, 0.0, True, 1)

# Auto

In [5]:
def Auto_gui(dat,loaded_manual_offset,TGT_save):
    font_small = ("gothic", 12)
    font_heading = ("gothic", 14)
    selected_mode = 0 #will determine the initial pose of the robot
    man_offset = 0.0 
    cluster_idx = 0
    selected_motion = 0  #default auto
    iterations = 1 #default
    Exit_flag = 0

    desc = ["Default robot pose with camera facing down towards the object",
            "Robot extended to the highest point and camera facing down towards the object.",
            "Robot facing forward and camera at a very low position. Object right in front of the robot.",
            "Robot facing forward and camera at a higher position. Object right in front of the robot.",
            "Robot and camera looking towards LEFT at a LOW position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot and camera looking towards LEFT at a HIGHER position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot EXTENDED and camera looking towards LEFT at a LOW position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot EXTENDED and camera looking towards LEFT at a HIGHER position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot and camera looking towards RIGHT at a LOW position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot and camera looking towards RIGHT at a HIGHER position. Object NOT ALIGNED to robot's base. Manual offset required (Horizontal Distance between robot's base and object).",
            "Robot EXTENDED and camera looking towards RIGHT at a LOW position. Object is aligned to robot's base (Horizontal distance is 0).",
            "Robot EXTENDED and camera looking towards RIGHT at a HIGHER position. Object is aligned to robot's base (Horizontal distance is 0).",
           ]
    
    while len(desc) < len(dat):
        desc.append("N/A") #add empty descriptions if more imgs added.
    
    
    file_list_column = [[sg.Text("Select a Pose that best describes your case.", font=font_heading),]]
    for i in range (0,len(dat),2):
        #print(i)
        file_list_column.append( [sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i), image_source=dat[i], 
                                     image_size = (None, None), image_subsample = 5, ), 
                                  sg.Button(button_text = "",enable_events = True, 
                                     size = (16, 4),key = 'but'+str(i+1), image_source=dat[i+1], 
                                     image_size = (None, None), image_subsample = 5, )] )

    
    buttons_layout = [
        [sg.Radio(text = "Auto select motion Path",font=font_small,default = True, enable_events = True, 
                  tooltip ='Program will decide motion Path',key = 'Rad_Auto',
                  auto_size_text = True, group_id=0, pad = ((5, 68), (0, 0))), 
         sg.Text("Enter Horizontal distance between robot's base and the Object:", size=(53, 1)
                            , font=font_small, expand_y=True), sg.Input(default_text = 0.0, 
                         size=(10, 1),key = "manual_offset",tooltip = "(in Meters)",disabled = True )],
        
        [sg.Radio(text = "Motion along X-axis",font=font_small, default = False, enable_events = True, 
                  tooltip ="Robot's motion along the camera's X axis",key = 'Rad_X',
                  auto_size_text = True, group_id=0 ), 
         sg.Text("", size=(43, 1), auto_size_text = True , font=font_small, expand_y=True), 
         sg.Text("Number of Iterations:", size=(19, 1), font=font_small, expand_y=True), 
         sg.Input(default_text = 1, size=(10, 1),key = "iterations",
            tooltip = "Enter the number of times you would like the robot to scan the surface")],
        
        [sg.Radio(text = "Motion along Y-axis", font=font_small, default = False, enable_events = True, 
                  tooltip ="Robot's motion along the camera's Y axis",key = 'Rad_Y',
                  auto_size_text = True, group_id=0, size=(65, 1) ),
         sg.Text("Save Targets?", size=(14, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_save, key ="TGT_save",pad=((0, 0),(0,5))
                  , tooltip = "Save Generated Targets for later use.")],
        
        [sg.Text("Default cluster index:", size=(19, 1), pad=((30, 0),(0,5)), font=font_small, expand_y=True), 
         sg.Input(default_text = 0, size=(10, 1),key = "cluster_idx",
            tooltip = "Cluster index to choose if there are more than 1 available"), 
         sg.Text("Hide all Previews after first iteration?", size=(31, 1), auto_size_text = True, 
         font=font_small, expand_y=True, pad=((184, 0),(0,5))), 
         sg.Checkbox("", default = True, key ="Hide_prev",pad=((0, 0),(0,5))
                  , tooltip = "Don't show previews. Fully automate scannig.")],
        
         
         [sg.Button(button_text = "Cancel",enable_events = True, tooltip ='First Cluster will be auto-selected', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((250, 0), (10, 0)), 
                                     image_size = (None, None), image_subsample = None, ),
         sg.Button(button_text = "OK",enable_events = True, disabled = True, tooltip ='Select atleast one cluster',
                                     size = (10, 2),key = 'OK', image_source=None, pad = ((20, 0), (10, 0)),
                                     image_size = (None, None), image_subsample = None )] ]

   
    image_viewer_column = [
        [sg.Text("Preview", font = font_heading)],
        [sg.Text("Description: " + desc[0],size=(100, 2),font=font_small, key="-TOUT-")],
        [sg.Image(key="img",source=dat[0])], [sg.HSeparator(pad=(0,10))], [sg.Column(buttons_layout)]]

    
    
    layout = [ [ sg.Column(file_list_column,size = (None, None), 
                           scrollable=True, vertical_scroll_only=True,expand_y=True ), 
                sg.VSeparator(), 
                sg.Column(image_viewer_column), ] 
             ]

    window = sg.Window("Auto Mode: Setup all initial values", layout)

    while True: # Run the Event Loop
        event, values = window.read()
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        if event == "Cancel" or event == sg.WIN_CLOSED:
            selected_mode = 0 #default Panda pose
            selected_motion = 0 #default
            iterations = 1 #default number of iters.
            Exit_flag = 1 #EXit program
            Hide_prev = window["Hide_prev"].get()
            break

        elif event_name == "but":
            try:
                window["img"].update(source=dat[event_idx])
            except:
                #print("Error")
                pass

            window["but"+str(selected_mode)].update(text = "",  image_subsample = 5) #clear last one
            selected_mode = event_idx
            window["but"+str(selected_mode)].update(text = "SELECTED",  image_subsample = 5)
            window["-TOUT-"].update(value = "Description: " + desc[selected_mode])
            window['OK'].set_tooltip("You may Proceed!")
            window["OK"].update(disabled = False)               
            #print(selected_mode)
            
            if selected_mode == 4 or selected_mode == 5 or selected_mode == 8 or selected_mode == 9:
                window["manual_offset"].update(value = loaded_manual_offset, disabled = False, move_cursor_to = "end")
                window["manual_offset"].set_tooltip("Value loaded from saved settings. (in Meters)")
            else:
                window["manual_offset"].update(value = 0.0, disabled = True)
                window["manual_offset"].set_tooltip("Not required. Setting to 0")

        elif event == "OK":
            
            man_offset = window["manual_offset"].get()
            TGT_save = window["TGT_save"].get()
            cluster_idx = window["cluster_idx"].get()
            iterations = window["iterations"].get()
            Hide_prev = window["Hide_prev"].get()
            
            #print("The current selection includes:",selected_mode)
            break

        elif event == "Rad_Auto":
            selected_motion = 0 #default
        elif event == "Rad_X":  #curve along Y-axis and motion along X axis
            selected_motion = 2 
        elif event == "Rad_Y":  #curve along X-axis and motion along Y axis
            selected_motion = 3 

    window.close()

    return selected_mode, float(man_offset), int(cluster_idx), selected_motion, TGT_save, int(iterations), Hide_prev, Exit_flag


dat=[]  #to hold images
loaded_manual_offset = 0.1
TGT_save = True
for f in sorted(glob.iglob("VI_appdata/Robo_object_positions/*")):
    im = Image.open(f)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    dat.append(data)


new_one = Auto_gui(dat,loaded_manual_offset,TGT_save)
new_one

(0, 0.0, 0, 0, True, 1, True, 1)

# Replay

In [ ]:
def Replay_gui(file_path, X_offset, Y_offset, Z_offset, TGT_save):
    font_small = ("gothic", 12)
    font_heading = ("gothic", 14)
    OK_flag = 0
    
    
    input_layout = [
        [sg.Text("Select Coordinate file:",font=font_heading)], 
        [sg.Input(default_text = file_path, key='file_path', visible=True, enable_events=True,readonly=True,), 
         sg.FileBrowse(file_types = (("Coordinate file","*.npy"),))],
        
        [sg.Text("The Offsets are added with respect to the World Coordinate system (wrt Robot's base)", 
                  auto_size_text = True, size=(50, 2), font=font_small, expand_y=True)],
        [sg.Text("")],
        [sg.Text("Add offset to X:", size=(25, 1), font=font_small, expand_y=True), 
         sg.Input(default_text = X_offset, size=(10, 1),key = "X_offset",tooltip = "(in Meters)" )],

        [sg.Text("Add offset to Y:", size=(25, 1), font=font_small, expand_y=True), 
         sg.Input(default_text = Y_offset, size=(10, 1),key = "Y_offset",tooltip = "(in Meters)" )],

        [sg.Text("Add offset to distance between camera and object:", size=(25, 2), font=font_small, expand_y=True), 
         sg.Input(default_text = Z_offset, size=(10, 1),key = "Z_offset",
              tooltip = "This will  OFFSET the distance between the camera and the object (in Meters)" )],
        
        [sg.Text("Save Targets?", size=(25, 1), auto_size_text = True , font=font_small, expand_y=True)
                    , sg.Checkbox("", default = TGT_save, key ="TGT_save",pad=((0, 0),(0,5))
                  , tooltip = "Save Generated Targets for later use.")]
                 ]
    

    buttons_layout = [
        
        [sg.Button(button_text = "Cancel",enable_events = True, tooltip ='Cancel', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((45, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,
         
         sg.Button(button_text = "Preview",enable_events = True, disabled = True, tooltip ='Preview in Rviz',
                                     size = (10, 2),key = 'Preview', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None ),
         
         sg.Button(button_text = "OK",enable_events = True, disabled = True, tooltip ='Select Coordinate file first!',
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]
                 ]
    

    layout = [
                [input_layout], 
                [sg.HSeparator(pad=(0,10))],
                [buttons_layout] 
        
             ]

    window = sg.Window("Replay Mode: Coordinate Selector.", layout)
    window.timer_start(200, key='timer', repeating=False) #used to refresh events
    
    event, values = window.read()
    file_path = window["file_path"].get()    
    if file_path.find('.npy') > -1:
        window["Preview"].update(disabled = False)
        window["OK"].update(disabled = False)

    
    while True: # Run the Event Loop
        
        event, values = window.read()
        #print( event, values)
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        
        if event == "Cancel" or event == sg.WIN_CLOSED:
            OK_flag = -1 #exit program
            
            break
            
        elif event == "file_path":
            window["file_path"].update(move_cursor_to = "end")
            file_path = window["file_path"].get()    
            if file_path.find('.npy') > -1:
                window["Preview"].update(disabled = False)
                window["OK"].update(disabled = False)
        
        
        elif event == "Preview" or event == sg.WIN_CLOSED:
            
            OK_flag = 0 #preview flag
            
            file_path = window["file_path"].get()
            X_offset = window["X_offset"].get()
            Y_offset = window["Y_offset"].get()
            Z_offset = window["Z_offset"].get()
            TGT_save = window["TGT_save"].get()
            
            break
        elif event == "OK":
            OK_flag = 1 #preview flag
            
            file_path = window["file_path"].get()
            X_offset = window["X_offset"].get()
            Y_offset = window["Y_offset"].get()
            Z_offset = window["Z_offset"].get()
            TGT_save = window["TGT_save"].get()
            break

    window.close()

    return file_path, float(X_offset), float(Y_offset), float(Z_offset), TGT_save, int(OK_flag)



#file_path='/home/shadowd/ws_panda/src/Visual_Inspection/VI_appdata/Saved_coordinates/saved.npy'
file_path='Select valid Coordinate file'
X_offset=0.0
Y_offset=0.0
Z_offset=0.0
TGT_save=True

new_one = Replay_gui(file_path, X_offset, Y_offset, Z_offset, TGT_save)




# Save

In [3]:
def Save_gui():
    font_small = ("gothic", 12)
    font_heading = ("gothic", 14)
    Save_mode = 0
    file_path='Select valid Coordinate file'
    
    
    radio_layout = [
        [sg.Radio(text = "Create a new file",default = True, enable_events = True, 
                  tooltip ="New file will be created based on mode of operation. If file exists, it will select different name",
                  key = 'Create_New_file', auto_size_text = True, group_id=0 , font=font_small )],

        [sg.Radio(text = "Overwrite previously saved file",default = False, enable_events = True, 
                  tooltip ='Warning... Coordinate file will be overwritten!',key = 'Overwrite_file',
                  auto_size_text = True, group_id=0, font=font_small )],

        [sg.Radio(text = "Append to a specific file",default = False, enable_events = True, 
                  tooltip ='Select a file to append new coordinates to...',key = 'Append_file',
                  auto_size_text = True, group_id=0, font=font_small )],

         ]

    input_layout = [        
        [sg.Text("Select Coordinate file:",font=font_heading, visible=False, key='file_txt')], 
        [sg.Input(default_text = file_path, key='file_path', visible=False, enable_events=True,readonly=True,), 
         sg.FileBrowse(file_types = (("Coordinate file","*.npy"),), visible=False, key='file_brw')]
                   ]

    buttons_layout = [
        
        [sg.Button(button_text = "Cancel",enable_events = True, tooltip ='Cancel', 
                                     size = (10, 2),key = 'Cancel', image_source=None,
                                     pad = ((45, 0), (0, 0)), 
                                     image_size = (None, None), image_subsample = None, ) ,

         sg.Button(button_text = "OK",enable_events = True, disabled = False, tooltip ='Save',
                                     size = (10, 2),key = 'OK', image_source=None,
                                     pad = ((20, 0), (0, 0)),
                                     image_size = (None, None), image_subsample = None )]
                 ]
    

    layout = [
                [radio_layout], 
                [input_layout],
                [sg.HSeparator(pad=(0,10))],
                [buttons_layout]
             ]

    window = sg.Window("Save Coordinates", layout)
    
    while True: # Run the Event Loop
        
        event, values = window.read()
        #print( event, values)
        try:
            event_name = event.rstrip('0123456789')
            event_idx = int(event[len(event_name):])
            #print(event_name, event_idx)
        except:
            pass

        
        if event == "Cancel" or event == sg.WIN_CLOSED:
            Save_mode = -1 #exit program
            
            break
            
        elif event == "file_path":
            window["file_path"].update(move_cursor_to = "end")
            file_path = window["file_path"].get()    
            if file_path.find('.npy') > -1:
                window["OK"].update(disabled = False)
        
        
        elif event == "OK":            
            file_path = window["file_path"].get()
            break

            
        elif event == "Create_New_file":
            Save_mode = 0 #default
        elif event == "Overwrite_file":
            Save_mode = 1 
        elif event == "Append_file":
            Save_mode = 2
            window["file_txt"].update(visible=True)
            window["file_path"].update(visible=True)
            window["file_brw"].update(visible=True)
            file_path = window["file_path"].get()    
            if file_path.find('.npy') > -1:
                window["OK"].update(disabled = False)
            else:
                window["OK"].update(disabled = True)

    window.close()

    return file_path, Save_mode



new_one = Save_gui()

new_one


('Select valid Coordinate file', -1)